In [34]:
import sys
path = "../../.."
if path not in sys.path:
    sys.path.insert(0, path)

In [35]:
from data_retrieval import lipade_groundtruth
from PIL import Image
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from lavis.models import load_model_and_preprocess
import numpy as np
import torch

sourceText = 'blip2'

device = "cpu" # torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [36]:
blip_model, blip_vis_processors, blip_text_processors = load_model_and_preprocess(name="blip_feature_extractor", model_type="base", is_eval=True, device=device)
blip2_model, blip2_vis_processors, blip2_text_processors = load_model_and_preprocess(name="blip2_feature_extractor", model_type="pretrain", is_eval=True, device=device)

/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/samuel-g/.pyenv/versions/3.11.11/lib/python3.11/site-packages/lavis/models/blip_models/blip.py:33: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they ar

In [37]:
corpus = "lipade_groundtruth"
rawPath = "../results/raw/" + corpus + "/"
distancePath = "../results/distance/" + corpus + "/"

In [38]:
_,m,_ = lipade_groundtruth.getDataset(mode="similar")

In [39]:
blip_images_embeddings = []
blip2_images_embeddings = []

for caption in tqdm(m[1]):
    blip_sample = {"text_input": [blip_text_processors["eval"](caption)]}
    blip2_sample = {"text_input": [blip2_text_processors["eval"](caption)]}

    blip_images_embeddings.append(blip_model.extract_features(blip_sample, mode="text").text_embeds[:,0,:].cpu().data.numpy())
    blip2_images_embeddings.append(blip2_model.extract_features(blip2_sample, mode="text").text_embeds[:,0,:].cpu().data.numpy())

100%|██████████| 279/279 [00:13<00:00, 21.23it/s]


In [40]:
blip_images_embeddings = np.array(blip_images_embeddings).reshape((len(m[0]), -1))
blip2_images_embeddings = np.array(blip2_images_embeddings).reshape((len(m[0]), -1))

In [41]:
blip_distance = 1 - (cosine_similarity(blip_images_embeddings, blip_images_embeddings)+1)/2
blip_distance -= np.diag(blip_distance)

blip2_distance = 1 - (cosine_similarity(blip2_images_embeddings, blip2_images_embeddings)+1)/2
blip2_distance -= np.diag(blip2_distance)

In [42]:
np.save(rawPath + sourceText + "_text_into_" + "blip.npy", blip_images_embeddings)
np.save(distancePath + sourceText + "_text_into_" + "blip.npy", blip_distance)

np.save(rawPath + sourceText + "_text_into_" + "blip2.npy", blip2_images_embeddings)
np.save(distancePath + sourceText + "_text_into_" + "blip2.npy", blip2_distance)